In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install aicrowd-cli pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... - \ | / done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.0 MB/s eta 0:00:00
  Created wheel for pyzmq: filename=pyzmq-22.1.0-cp310-cp310-linux_x86_64.whl size=538912 sha256=11b271ffa9b394f928d5871fbca59b92d56d5e33322080bdac0506dbc0b6e471
  Stored in directory: /root/.cache/pip/wheels/2d/e1/f6/45f17238de3475af44fca25559ef8d2f70f2b230d941da499e
Successfully built pyzmq
  Attempting uninstall: semver
    Found existing inst

In [3]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/-8XbvqqjrSDnZNVFiQU7P0zTU-bTQEQhb5atU5lEAac
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/-8XbvqqjrSDnZNVFiQU7P0zTU-bTQEQhb5atU5lEAac'
API Key valid
Gitlab access token valid
Saved details successfully!


In [4]:
!aicrowd dataset download --challenge task-1-next-product-recommendation

sessions_test_task1_phase1.csv:   0%|               | 0.00/19.4M [00:00<?, ?B/s]
sessions_test_task1_phase1.csv:  22%|█▎    | 4.19M/19.4M [00:01<00:02, 5.16MB/s]
sessions_test_task1_phase1.csv:  38%|██▎   | 7.34M/19.4M [00:01<00:01, 8.91MB/s]
sessions_test_task2_phase1.csv: 100%|██████| 1.92M/1.92M [00:00<00:00, 1.98MB/s]
sessions_test_task1_phase1.csv:  70%|████▏ | 13.6M/19.4M [00:02<00:00, 9.34MB/s]
sessions_test_task3_phase1.csv:   0%|               | 0.00/2.67M [00:00<?, ?B/s]
sessions_test_task3_phase1.csv:  39%|██▎   | 1.05M/2.67M [00:00<00:01, 1.24MB/s]
sessions_test_task3_phase1.csv: 100%|██████| 2.67M/2.67M [00:01<00:00, 2.56MB/s]
sessions_test_task1_phase1.csv: 100%|██████| 19.4M/19.4M [00:03<00:00, 5.41MB/s]
sessions_test_task1.csv:   0%|                      | 0.00/19.3M [00:00<?, ?B/s]
sessions_test_task1.csv:  11%|█▍           | 2.10M/19.3M [00:01<00:10, 1.65MB/s]
sessions_test_task2.csv:  55%|███████      | 1.05M/1.91M [00:00<00:00, 1.21MB/s]
sessions_test_task2.csv: 100

In [5]:
import os
import numpy as np
import pandas as pd
from functools import lru_cache

In [6]:
train_data_dir = '.'
test_data_dir = '.'
task = 'task1'
PREDS_PER_SESSION = 100

In [7]:
def read_product_data():
    return pd.read_csv(os.path.join(train_data_dir, 'products_train.csv'))
def read_train_data():
    return pd.read_csv(os.path.join(train_data_dir, 'sessions_train.csv'))
def read_test_data(task):
    return pd.read_csv(os.path.join(test_data_dir, f'sessions_test_{task}.csv'))

In [8]:
products = read_product_data()
products.sample(5)

,id,locale,title,price,brand,color,size,model,material,author,desc
1178693,B01G2GFIXY,UK,Dudu N Girlie Fitted Cot Bed Sheets | Cot Shee...,6.85,DUDU N GIRLIE,Pink,60x120 cm (Pack of 1),B01G2GFIXY,100% Cotton,NaN,ALL-NIGHT SUPPORT - Travel Cot Sheets 120x60 F...
128643,B00EMGLQS8,DE,Ketozolin 2% Shampoo bei seborrhoischer Dermat...,16.49,DERMAPHARM AG,NaN,120 ml (1er Pack),Y4,NaN,NaN,"DERMAPHARM AG Arzneimittel, Deutschland"
1437195,B09R1STXZJ,ES,Finocam - Agenda 2022 2023 Secundaria Semana V...,3.69,"Cabero Group 1916, S.A",Imposible,PEQUEÑO - 8º - 120x164 mm,1,Papel,NaN,Contenido: La anualidad incluye la información...
246940,B095PQ354G,DE,"SAUBA LOOWY | Toilettenbürste ohne Borsten, Se...",25.90,SAUBA,Schwarz/Grau-schwarz,NaN,schwarz matt,NaN,NaN,Antibakterielle Wirkung des Bürstenkopfes (TÜV...
429663,B09Z6Z6PM8,DE,NatureMark Basic Bettlaken - weiches und pfleg...,11.99,NatureMark,Weiß,140 x 200 cm,NaN,100% Baumwolle,NaN,Ein Markenprodukt von NatureMark vielseitig ve...


In [9]:
train_sessions = read_train_data()
train_sessions.head(5)

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [10]:
test_sessions = read_test_data(task)
test_sessions.head(5)

,prev_items,locale
0,['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV'],DE
1,['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF'],DE
2,['B09Z4T2GJ3' 'B09Z3FBXMB' 'B0936K9LTJ' 'B09Z4...,DE
3,['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y1G5F3Y'],DE
4,['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0B2DRKZ6X'],DE


In [11]:
test_sessions_UK = pd.DataFrame()
products_UK = pd.DataFrame()

In [12]:
train_sessions_UK = pd.DataFrame()
products_UK = pd.DataFrame()

In [13]:
# Filter train_sessions for rows with locale='UK'
train_sessions_UK = train_sessions[train_sessions['locale'] == 'DE']

In [14]:
test_sessions_UK.shape

(0, 0)

In [15]:
test_sessions_UK = test_sessions[test_sessions['locale'] == 'DE']

In [16]:
products_UK = products[products['locale'] == 'DE']

In [17]:
train_sessions_UK.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [18]:
train_sessions_UK

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
1111411,['B06X9BB2D7' 'B09RWWGXZJ'],B09RWWWYGZ,DE
1111412,['B0BK2WGCN4' 'B08H93ZRLL' 'B0BK2WGCN4' 'B08H9...,B001BWJEXK,DE
1111413,['B08X1SDBLB' 'B06WVZRBJ1'],B09YRTCM8X,DE
1111414,['B0B8NNHR5N' 'B0BBCCB2S1' 'B09R222SDP' 'B0B5F...,B07XJ3H1RM,DE


In [19]:
dat_train = train_sessions_UK['prev_items'].tolist()
dat_test = test_sessions_UK['prev_items'].tolist()

def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

cleaned_train = []
for i in range(len(dat_train)):
    
    cleaned_train.append(str2list(dat_train[i]))
#cleaned_train


cleaned_test = []
for i in range(len(dat_test)):
    
    cleaned_test.append(str2list(dat_test[i]))
#cleaned_test


train_sessions_UK['prev_items'] = cleaned_train
test_sessions_UK['prev_items'] = cleaned_test

/tmp/ipykernel_20/239052266.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sessions_UK['prev_items'] = cleaned_train
/tmp/ipykernel_20/239052266.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sessions_UK['prev_items'] = cleaned_test


In [20]:
# # Convert strings in 'prev_items' to lists
# train_sessions_UK['prev_items'] = train_sessions_UK['prev_items'].apply(lambda x: x[1:-1].split(' '))
# train_sessions_UK['prev_items'] = train_sessions_UK['prev_items'].apply(lambda x: [item.strip().strip("'") for item in x])

In [21]:
# # Convert strings in 'prev_items' to lists
# test_sessions_UK['prev_items'] = test_sessions_UK['prev_items'].apply(lambda x: x[1:-1].split(' '))
# test_sessions_UK['prev_items'] = test_sessions_UK['prev_items'].apply(lambda x: [item.strip().strip("'") for item in x])

In [22]:
min_count = test_sessions_UK['prev_items'].apply(lambda x: len(x)).min()


In [23]:
train_sessions_UK['prev_items'] = train_sessions_UK.apply(lambda row: row['prev_items'] + [row['next_item']], axis=1)


/tmp/ipykernel_20/4290034151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sessions_UK['prev_items'] = train_sessions_UK.apply(lambda row: row['prev_items'] + [row['next_item']], axis=1)


In [24]:
!pip install nvtabular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / 

In [25]:
import os
import numpy as np 
import gc
import shutil
import glob
import nvtabular as nvt

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [26]:
# Assuming this is your dataframe

# Adding session_id based on index
train_sessions_UK.reset_index(level=0, inplace=True)
train_sessions_UK.rename(columns={'index':'session_id'}, inplace=True)
# Split 'prev_items' into separate rows
train_sessions_UK = train_sessions_UK.explode('prev_items')



/tmp/ipykernel_20/1747550676.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sessions_UK.rename(columns={'index':'session_id'}, inplace=True)


In [27]:
# Assuming this is your dataframe

# Adding session_id based on index
test_sessions_UK.reset_index(level=0, inplace=True)
test_sessions_UK.rename(columns={'index':'session_id'}, inplace=True)
# Split 'prev_items' into separate rows
test_sessions_UK = test_sessions_UK.explode('prev_items')

/tmp/ipykernel_20/1710534608.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sessions_UK.rename(columns={'index':'session_id'}, inplace=True)


In [28]:
train_sessions_UK.head()

,session_id,prev_items,next_item,locale
0,0,B09W9FND7K,B09M7GY217,DE
0,0,B09JSPLN1M,B09M7GY217,DE
0,0,B09M7GY217,B09M7GY217,DE
1,1,B076THCGSG,B001B4THSA,DE
1,1,B007MO8IME,B001B4THSA,DE


In [29]:
train_sessions_UK['session_id'] = train_sessions_UK['session_id']+1000000

In [30]:
test_sessions_UK.head()

,session_id,prev_items,locale
0,0,B087VLP2RT,DE
0,0,B09BRQSHYH,DE
0,0,B099KW4ZLV,DE
1,1,B08XW4W667,DE
1,1,B096VMCJYF,DE


In [31]:
# Merging with 'products' dataframe to get the product details and dropping 'id' and 'locale' columns
session_attribute_train = pd.merge(train_sessions_UK, products_UK, left_on='prev_items', right_on='id', how='left').drop(['id', 'locale_x','locale_y'], axis=1)




In [32]:
# Merging with 'products' dataframe to get the product details and dropping 'id' and 'locale' columns
session_attribute_test = pd.merge(test_sessions_UK, products_UK, left_on='prev_items', right_on='id', how='left').drop(['id', 'locale_x','locale_y'], axis=1)




In [33]:
session_attribute_train.head(5)

,session_id,prev_items,next_item,title,price,brand,color,size,model,material,author,desc
0,1000000,B09W9FND7K,B09M7GY217,OREiN LED Einbaustrahler 230V Dimmbar & IP65 L...,39.99,OREiN,"6*ip65 (3000k,dimmbar)",Lochgröße Φ68-72mm,NaN,塑料,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
1,1000000,B09JSPLN1M,B09M7GY217,OREiN LED Spot LED Einbaustrahler 230V IP65 LE...,36.99,OREiN,6*ip65(3000k),Lochgröße Φ68-72mm,NaN,Kunststoff,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
2,1000000,B09M7GY217,B09M7GY217,"Samsung Galaxy Buds2, Kabellose Kopfhörer, Wir...",109.99,Samsung,White,NaN,Samsung Galaxy Buds2,Silikon,NaN,4 Farben zur Auswahl – Graphite. White. Olive....
3,1000001,B076THCGSG,B001B4THSA,"VIETSCHI Profi DIN Plus 12,5L weiß - doppeldec...",39.80,Vietschi,NaN,12.5 l (1er Pack),NaN,NaN,NaN,"✅ Einfache Verarbeitung - Egal ob Streichen, R..."
4,1000001,B007MO8IME,B001B4THSA,"Caparol Capa DIN 12,500 L Wandfarben",40.27,Caparol,Weiss,12.5 l (1er Pack),11283000,Dispersionsfarbe,NaN,Caparol


In [34]:
session_attribute_test.head(5)

,session_id,prev_items,title,price,brand,color,size,model,material,author,desc
0,0,B087VLP2RT,Xbox Wireless Controller Robot White,57.99,Microsoft,Robot White,NaN,QAS-00002,NaN,NaN,"An der 3,5mm-Stereo-Headsetbuchse kannst du ko..."
1,0,B09BRQSHYH,Xbox Wireless Controller Aqua Shift + Xbox Pla...,40000000.07,Microsoft,Aqua Shift,NaN,QAU-00027,NaN,NaN,NaN
2,0,B099KW4ZLV,Xbox Wireless Controller Aqua Shift,107.83,Microsoft,Aqua Shift,NaN,QAU-00027,NaN,NaN,"An der 3,5mm-Stereo-Headsetbuchse kannst du ko..."
3,1,B08XW4W667,KREUL 23050 - Mucki Fingerfarbe für Glückskind...,9.79,Kreul,"Gelb, Pink, Blau, Grün, Orange und Silber",50 ml (6er Pack),10128657,NaN,NaN,Cremig-pastose Konsistenz für die optimale Anw...
4,1,B096VMCJYF,Marabu 0303000050800 - KiDS Fingerfarben-Set m...,6.40,Marabu,Mehrfarbig,35 ml (6er Pack),0303000050800,unbekannt,NaN,"Cremige, leicht verstreichbare Fingermalfarbe ..."


In [35]:
raw_df1 = session_attribute_train.drop('next_item',axis = 1)

In [36]:
# session_attribute_test = session_attribute_test.drop('event_time_dt-first',axis = 1)

In [37]:
# session_attribute_test['event_time_dt-first'] = 2

In [38]:
# raw_df1['event_time_dt-first'] = 1

In [39]:
np.min(raw_df1.session_id)

1000000

In [40]:
np.max(session_attribute_test['session_id'])

104567

In [41]:
session_attribute_test.shape

(447533, 11)

In [42]:
join_data = pd.concat([raw_df1, session_attribute_test])

In [43]:
join_data.sample(5)

,session_id,prev_items,title,price,brand,color,size,model,material,author,desc
4434707,1828786,B0B2PQLWD9,"Philips Hue Tap Dial switch Drehschalter, weiß...",42.99,Philips,Weiß,NaN,929003500101,Kunststoff,NaN,Flexibel einsetzbar: Stellen Sie Lichtszenen i...
3502856,1654447,B01NALRBE5,"deleyCON 1,5m SAT Antennenkabel HDTV Satellite...",9.99,deleyCON,Schwarz,1.5 M,MK1813,NaN,NaN,Anschluss: F-Stecker zu F-Stecker (Koaxial) //...
1344168,1251082,B09BHQ54PG,"Lichterkette Batterie 2 m, 12 Stück 20 Led Lic...",16.99,HELESIN,Mehrfarbig Led Lichterkette,12 pack,NaN,NaN,NaN,【Breite Anwendung】 Schaffen Sie eine romantisc...
4789201,1895081,B09C62L43D,"Philips Hue Wandschalter Modul, Doppelpack, er...",57.84,Philips Hue,Weiß,2-er Pack,929003017102,Kunststoff,NaN,Einfache Integration in Ihr Hue System: Der Wa...
1724094,1322346,B09V4BH8B2,Sidorenko Schreibtischunterlage PU Leder 80x40...,15.99,Sidorenko,Schwarz,80x40 cm,NaN,Kunstleder,NaN,✅ ANGENEHMES ARBEITEN - Die hochwertige Schrei...


In [44]:
raw_df = join_data

In [45]:
# raw_df = raw_df1[0:1000]

In [46]:
raw_df.head()

,session_id,prev_items,title,price,brand,color,size,model,material,author,desc
0,1000000,B09W9FND7K,OREiN LED Einbaustrahler 230V Dimmbar & IP65 L...,39.99,OREiN,"6*ip65 (3000k,dimmbar)",Lochgröße Φ68-72mm,NaN,塑料,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
1,1000000,B09JSPLN1M,OREiN LED Spot LED Einbaustrahler 230V IP65 LE...,36.99,OREiN,6*ip65(3000k),Lochgröße Φ68-72mm,NaN,Kunststoff,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
2,1000000,B09M7GY217,"Samsung Galaxy Buds2, Kabellose Kopfhörer, Wir...",109.99,Samsung,White,NaN,Samsung Galaxy Buds2,Silikon,NaN,4 Farben zur Auswahl – Graphite. White. Olive....
3,1000001,B076THCGSG,"VIETSCHI Profi DIN Plus 12,5L weiß - doppeldec...",39.80,Vietschi,NaN,12.5 l (1er Pack),NaN,NaN,NaN,"✅ Einfache Verarbeitung - Egal ob Streichen, R..."
4,1000001,B007MO8IME,"Caparol Capa DIN 12,500 L Wandfarben",40.27,Caparol,Weiss,12.5 l (1er Pack),11283000,Dispersionsfarbe,NaN,Caparol


In [47]:
cols = list(raw_df.columns)
cols.remove('session_id')
cols

# load data 
df_event = nvt.Dataset(raw_df) 

# categorify user_session 
cat_feats = ['session_id'] >> nvt.ops.Categorify()

workflow = nvt.Workflow(cols + cat_feats)
workflow.fit(df_event)
df = workflow.transform(df_event).to_ddf().compute()

/opt/conda/lib/python3.10/site-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [48]:
# # check out the columns with nulls
# raw_df.isnull().any()

In [49]:
# categorify features 
item_id = ['prev_items'] >> nvt.ops.TagAsItemID()
cat_feats = item_id + ['title','price','brand','color','size','model','material','author','desc'] >> nvt.ops.Categorify(start_index=1)

In [50]:
df.head()

,session_id,prev_items,title,price,brand,color,size,model,material,author,desc
0,958284,B09W9FND7K,OREiN LED Einbaustrahler 230V Dimmbar & IP65 L...,39.99,OREiN,"6*ip65 (3000k,dimmbar)",Lochgröße Φ68-72mm,NaN,塑料,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
1,958284,B09JSPLN1M,OREiN LED Spot LED Einbaustrahler 230V IP65 LE...,36.99,OREiN,6*ip65(3000k),Lochgröße Φ68-72mm,NaN,Kunststoff,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
2,958284,B09M7GY217,"Samsung Galaxy Buds2, Kabellose Kopfhörer, Wir...",109.99,Samsung,White,NaN,Samsung Galaxy Buds2,Silikon,NaN,4 Farben zur Auswahl – Graphite. White. Olive....
3,424482,B076THCGSG,"VIETSCHI Profi DIN Plus 12,5L weiß - doppeldec...",39.80,Vietschi,NaN,12.5 l (1er Pack),NaN,NaN,NaN,"✅ Einfache Verarbeitung - Egal ob Streichen, R..."
4,424482,B007MO8IME,"Caparol Capa DIN 12,500 L Wandfarben",40.27,Caparol,Weiss,12.5 l (1er Pack),11283000,Dispersionsfarbe,NaN,Caparol


In [51]:
raw_df.head()

,session_id,prev_items,title,price,brand,color,size,model,material,author,desc
0,1000000,B09W9FND7K,OREiN LED Einbaustrahler 230V Dimmbar & IP65 L...,39.99,OREiN,"6*ip65 (3000k,dimmbar)",Lochgröße Φ68-72mm,NaN,塑料,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
1,1000000,B09JSPLN1M,OREiN LED Spot LED Einbaustrahler 230V IP65 LE...,36.99,OREiN,6*ip65(3000k),Lochgröße Φ68-72mm,NaN,Kunststoff,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...
2,1000000,B09M7GY217,"Samsung Galaxy Buds2, Kabellose Kopfhörer, Wir...",109.99,Samsung,White,NaN,Samsung Galaxy Buds2,Silikon,NaN,4 Farben zur Auswahl – Graphite. White. Olive....
3,1000001,B076THCGSG,"VIETSCHI Profi DIN Plus 12,5L weiß - doppeldec...",39.80,Vietschi,NaN,12.5 l (1er Pack),NaN,NaN,NaN,"✅ Einfache Verarbeitung - Egal ob Streichen, R..."
4,1000001,B007MO8IME,"Caparol Capa DIN 12,500 L Wandfarben",40.27,Caparol,Weiss,12.5 l (1er Pack),11283000,Dispersionsfarbe,NaN,Caparol


In [52]:
# Smoothing price long-tailed distribution and applying standardization
price_log = ['price'] >> nvt.ops.LogOp() >> nvt.ops.Normalize(out_dtype=np.float32) >> nvt.ops.Rename(name='price_log_norm')

In [53]:
from merlin.dag import ColumnSelector
from merlin.schema import Schema, Tags
# Relative price to the average price for the category_id
def relative_price_to_avg_categ(col, gdf):
    epsilon = 1e-5
    col = ((gdf['price'] - col) / (col + epsilon)) * (col > 0).astype(int)
    return col
    
avg_category_id_pr = ['brand'] >> nvt.ops.JoinGroupby(cont_cols =['price'], stats=["mean"]) >> nvt.ops.Rename(name='avg_category_id_price')
relative_price_to_avg_category = (
    avg_category_id_pr >> 
    nvt.ops.LambdaOp(relative_price_to_avg_categ, dependency=['price']) >> 
    nvt.ops.Rename(name="relative_price_to_avg_categ_id") >>
    nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)

In [54]:
groupby_feats = ['session_id'] + cat_feats + price_log + relative_price_to_avg_category

In [55]:
groupby_feats

<Node + output>

In [56]:
# from nvtabular.ops import LambdaOp
# # Define LambdaOp to change data type
# change_dtype = nvt.ops.LambdaOp(lambda col: col.astype('int64'), 'prev_items', 'prev_items')

In [57]:
price_log + relative_price_to_avg_category

# Define Groupby Workflow
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        'prev_items': ["list", "count"],
        'title': ["list"], 
        'price_log_norm': ["list"],
        'relative_price_to_avg_categ_id': ["list"],
        'brand': ["list"], 
        'color': ["list"], 
        'size': ["list"],
        'model': ["list"],
        'material': ["list"],
        'author': ["list"],
        'desc': ["list"]
        },
    name_sep="-")



In [58]:
# import nvtabular as nvt
# from nvtabular.ops import LambdaOp

# # Define the lambda function to change the data type
# change_dtype = LambdaOp(lambda col: col.astype('int64'), 'prev_items-list_seq', 'prev_items-list_seq')


In [59]:
groupby_features

<Node Groupby output>

In [60]:
groupby_features_list = groupby_features[
    'prev_items-list',
    'title-list',
    'price_log_norm-list',
    'relative_price_to_avg_categ_id-list',
    'brand-list',
    'color-list',
    'size-list',
    'model-list',
    'material-list',
    'author-list',
    'desc-list'
]

In [61]:
SESSIONS_MAX_LENGTH = 10
MINIMUM_SESSION_LENGTH = 2

In [62]:
groupby_features_trim = groupby_features_list >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH)

In [63]:
# # calculate session day index based on 'timestamp-first' column
# day_index = ((groupby_features['event_time_dt-first'])  >> 
#              nvt.ops.LambdaOp(lambda col: (col - col.min()) +1) >>
#              nvt.ops.Rename(f = lambda col: "day_index") >>
#              nvt.ops.AddMetadata(tags=[Tags.CATEGORICAL])
#             )

In [64]:
sess_id = groupby_features['session_id'] >> nvt.ops.AddMetadata(tags=[Tags.CATEGORICAL])
selected_features = sess_id + groupby_features['prev_items-count'] + groupby_features_trim

In [65]:
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["prev_items-count"] >= MINIMUM_SESSION_LENGTH)

In [66]:
print(filtered_sessions)

<Node Filter output>


In [67]:
workflow.input_schema

,name,tags,dtype,is_list,is_ragged
0,session_id,(),"DType(name='int64', element_type=<ElementType....",False,False
1,prev_items,(),"DType(name='object', element_type=<ElementType...",False,False
2,title,(),"DType(name='object', element_type=<ElementType...",False,False
3,price,(),"DType(name='float64', element_type=<ElementTyp...",False,False
4,brand,(),"DType(name='object', element_type=<ElementType...",False,False
5,color,(),"DType(name='object', element_type=<ElementType...",False,False
6,size,(),"DType(name='object', element_type=<ElementType...",False,False
7,model,(),"DType(name='object', element_type=<ElementType...",False,False
8,material,(),"DType(name='object', element_type=<ElementType...",False,False
9,author,(),"DType(name='object', element_type=<ElementType...",False,False


In [68]:
workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(df)
# Learn features statistics necessary of the preprocessing workflow
# The following will generate schema.pbtxt file in the provided folder and export the parquet files.


/opt/conda/lib/python3.10/site-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [69]:
workflow.fit_transform(dataset).to_parquet(os.path.join('/kaggle/working/', "processed_nvt"))

/opt/conda/lib/python3.10/site-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [70]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,prev_items-count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,1.0,.//categories/unique.prev_items.parquet,0.0,517473.0,prev_items,517474.0,512.0,NaN,NaN
2,prev_items-list,"(Tags.ID, Tags.LIST, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,1.0,.//categories/unique.prev_items.parquet,0.0,517473.0,prev_items,517474.0,512.0,0.0,10.0
3,title-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,1.0,.//categories/unique.title.parquet,0.0,494819.0,title,494820.0,512.0,0.0,10.0
4,price_log_norm-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0
5,relative_price_to_avg_categ_id-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0
6,brand-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,1.0,.//categories/unique.brand.parquet,0.0,72003.0,brand,72004.0,512.0,0.0,10.0
7,color-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,1.0,.//categories/unique.color.parquet,0.0,81566.0,color,81567.0,512.0,0.0,10.0
8,size-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,1.0,.//categories/unique.size.parquet,0.0,85966.0,size,85967.0,512.0,0.0,10.0
9,model-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,1.0,.//categories/unique.model.parquet,0.0,207011.0,model,207012.0,512.0,0.0,10.0


In [71]:
INPUT_DATA_DIR = '/kaggle/working/'

In [72]:
# read in the processed train dataset
sessions_gdf = pd.read_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))

In [73]:
print(sessions_gdf.head(2))

   session_id  prev_items-count  \
0           1               157   
1           2               130   

                                     prev_items-list  \
0  [192167, 229325, 229344, 192130, 229361, 22934...   
1  [19925, 105095, 225624, 152134, 193989, 464629...   

                                          title-list  \
0  [183114, 228358, 228299, 183036, 228287, 22826...   
1  [20800, 107486, 227654, 157211, 195946, 443700...   

                                 price_log_norm-list  \
0  [-0.03476342186331749, -0.06477032601833344, -...   
1  [-0.15433914959430695, -0.15832844376564026, -...   

                 relative_price_to_avg_categ_id-list  \
0  [-0.1540341383588659, -0.22870012042532817, -0...   
1  [0.10750393690019852, 0.09382523113994498, 0.0...   

                                          brand-list  \
0  [555, 555, 555, 555, 555, 555, 555, 555, 555, ...   
1  [3721, 3721, 3721, 9313, 32706, 32706, 55403, ...   

                       color-list  \
0  [2, 2, 2,

In [74]:
session_ids = pd.read_parquet('/kaggle/working/categories/unique.session_id.parquet')
item_ids = pd.read_parquet('/kaggle/working/categories/unique.prev_items.parquet')

In [75]:
# First, make sure the index of `session_ids` is integer type, this ensures the matching works correctly
session_ids.index = session_ids.index.astype(int)

# Then map `session_id` in `sessions_df` to index in `session_ids`
sessions_gdf['session_org'] = sessions_gdf['session_id'].map(session_ids['session_id'])


In [76]:
sessions_df_test = sessions_gdf[sessions_gdf['session_org'] < 999999]

In [77]:
sessions_gdf = sessions_gdf.drop('session_org', axis=1)
sessions_df_test = sessions_df_test.drop('session_org', axis=1)

In [78]:
# filtered_df_20 = sessions_gdf[sessions_gdf['prev_items-count'] >= 2]

In [79]:
# filtered_df_20.to_parquet('/kaggle/working/processed_nvt/part_0_filtered_2.parquet')

In [80]:
# filtered_df_10 = sessions_gdf[sessions_gdf['prev_items-count'] >= 5]
# filtered_df_10.to_parquet('/kaggle/working/processed_nvt/part_0_filtered_5.parquet')

In [81]:
sessions_df_test.to_parquet('/kaggle/working/processed_nvt/test_0.parquet')

In [82]:
len(sessions_gdf)

1215984

In [83]:
# pp

In [84]:
# import numpy as np
# import pandas as pd


# # Generate an array of random integers between 1 and 4
# random_numbers = np.random.randint(1, 4, sessions_gdf.shape[0])

# # Add the array as a new column in the DataFrame
# sessions_gdf['day_index'] = random_numbers
# sessions_gdf.head()

In [85]:
# !pip install transformers4rec

In [86]:
# PARTITION_COL = 'day_index'
# from transformers4rec.utils.data_utils import save_time_based_splits
# save_time_based_splits(data=nvt.Dataset(sessions_gdf),
#                        output_dir= '/kaggle/working/final1/',
#                        partition_col=PARTITION_COL,
#                        timestamp_col='session_id',
#                        test_size = 0.0,
#                        val_size = 0.1,
#                       )

In [87]:
# workflow_path = '/kaggle/working/workflow_etl')
# workflow.save(workflow_path)

# pamda


In [88]:
# ls

In [89]:
# cd processed_nvt/

In [90]:
# train_sessions_UK.head(5)

In [91]:
# import sys
# !cp /kaggle/input/rapids/rapids.0.17.0 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [92]:
# # Check CUDA/cuDNN Version
# !nvcc -V && which nvcc

In [93]:
# import cudf
# from cudf.core import Series
# import math

In [94]:
# import cudf
# import os

In [95]:
# # read in the processed train dataset
# sessions_gdf = cudf.read_parquet('/kaggle/working/processed_nvt/part_0.parquet')

In [96]:
# # read in the processed train dataset
# sessions_gdf = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))

In [97]:
# print(sessions_gdf.head())

In [98]:
# from IPython.display import FileLink
# FileLink(r'/kaggle/working/processed_nvt/part_0.parquet')

In [99]:
# cont_feats = groupby_features['et_dayofweek_sin-list', 'product_recency_days_log_norm-list'] >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])


In [100]:
# cd working/

In [101]:
!zip -r categories_DE_full_new.zip /kaggle/working/categories

  adding: kaggle/working/categories/ (stored 0%)
  adding: kaggle/working/categories/unique.model.parquet (deflated 46%)
  adding: kaggle/working/categories/unique.size.parquet (deflated 51%)
  adding: kaggle/working/categories/unique.author.parquet (deflated 49%)
  adding: kaggle/working/categories/unique.prev_items.parquet (deflated 58%)
  adding: kaggle/working/categories/unique.price.parquet (deflated 51%)
  adding: kaggle/working/categories/unique.title.parquet (deflated 59%)
  adding: kaggle/working/categories/unique.material.parquet (deflated 56%)
  adding: kaggle/working/categories/unique.desc.parquet (deflated 59%)
  adding: kaggle/working/categories/unique.color.parquet (deflated 52%)
  adding: kaggle/working/categories/unique.brand.parquet (deflated 41%)
  adding: kaggle/working/categories/unique.session_id.parquet (deflated 70%)
  adding: kaggle/working/categories/cat_stats.brand.parquet (deflated 36%)


In [102]:
!zip -r processed_DE_full_new.zip /kaggle/working/processed_nvt

  adding: kaggle/working/processed_nvt/ (stored 0%)
  adding: kaggle/working/processed_nvt/.merlin/ (stored 0%)
  adding: kaggle/working/processed_nvt/.merlin/schema.json (deflated 85%)
  adding: kaggle/working/processed_nvt/_metadata.json (deflated 27%)
  adding: kaggle/working/processed_nvt/part_0.parquet (deflated 47%)
  adding: kaggle/working/processed_nvt/_metadata (deflated 75%)
  adding: kaggle/working/processed_nvt/_file_list.txt (stored 0%)
  adding: kaggle/working/processed_nvt/test_0.parquet (deflated 17%)
  adding: kaggle/working/processed_nvt/schema.pbtxt (deflated 91%)


In [103]:
# cd workflow_etl/

In [104]:
# cd categories/


In [105]:
# # workflow_path = os.path.join('/kaggle/working/', 'workflow_etl')
# workflow.save(workflow_path)